In [ ]:
import os
import sys
from pathlib import Path
from typing import List, Optional
import gradio as gr
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_core.messages import HumanMessage
from langchain_huggingface import HuggingFaceEmbeddings
from unstructured.partition.auto import partition
from langchain_ollama import ChatOllama
from IPython.display import Markdown, display
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

# Assuming you have this module
sys.path.append("../../../llm_engineering")
from api_clients import create_clients


class RAGBookQA:
    """RAG-based Question Answering system for PDF books."""
    
    def __init__(
        self,
        db_name: str = "vector_db",
        embedding_model: str = "all-MiniLM-L6-v2",
        chunk_size: int = 1000,
        chunk_overlap: int = 200,
        temperature: float = 0.0,
        k_docs: int = 4
    ):
        """
        Initialize the RAG system.
        
        Args:
            db_name: Directory name for vector database
            embedding_model: HuggingFace model name for embeddings
            chunk_size: Size of text chunks for splitting
            chunk_overlap: Overlap between chunks
            temperature: LLM temperature for response generation
            k_docs: Number of documents to retrieve
        """
        self.db_name = db_name
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.temperature = temperature
        self.k_docs = k_docs
        
        # Initialize embeddings
        print("Loading embedding model...")
        self.embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
        
        # Initialize LLM
        print("Initializing LLM...")
        clients = create_clients()
        self.llm = ChatOllama(
            model=clients["models"]["OLLAMA_MODEL"],
            temperature=self.temperature,
        )
        
        self.vectorstore = None
        self.retriever = None
        
    def load_and_process_pdf(self, filename: str, force_reload: bool = False) -> None:
        """
        Load PDF, chunk it, and create vector store.
        
        Args:
            filename: Path to PDF file
            force_reload: If True, recreate vector store even if it exists
        """
        if not os.path.exists(filename):
            raise FileNotFoundError(f"File not found: {filename}")
        
        # Check if we need to reload
        if os.path.exists(self.db_name) and not force_reload:
            print(f"Loading existing vector store from {self.db_name}...")
            self.vectorstore = Chroma(
                persist_directory=self.db_name,
                embedding_function=self.embeddings
            )

            self.retriever = self.vectorstore.as_retriever(
                search_kwargs={"k": self.k_docs}
            )
            return
        
        # Delete existing collection if force reload
        if os.path.exists(self.db_name):
            print("Deleting existing vector store...")
            try:
                Chroma(
                    persist_directory=self.db_name,
                    embedding_function=self.embeddings
                ).delete_collection()
            except Exception as e:
                print(f"Warning: Could not delete existing collection: {e}")
        
        # Partition PDF
        print(f"Processing PDF: {filename}")
        try:
            file_partition = partition(filename)
        except Exception as e:
            raise RuntimeError(f"Failed to partition PDF: {e}")
        
        # Combine text with metadata
        text = '\n'.join([str(el) for el in file_partition])
        doc = Document(
            page_content=text,
            metadata={
                "source": filename,
                "filename": os.path.basename(filename)
            }
        )
        
        # Split into chunks (using RecursiveCharacterTextSplitter for better results)
        print("Splitting document into chunks...")
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size,
            chunk_overlap=self.chunk_overlap,
            separators=["\n\n", "\n", ". ", " ", ""] # it has hierarchical order
        )
        chunks = text_splitter.split_documents([doc])
        
        # Add chunk numbers to metadata
        for i, chunk in enumerate(chunks):
            chunk.metadata["chunk_id"] = i
            chunk.metadata["total_chunks"] = len(chunks)
            chunk.metadata["file_name"] = filename
        
        print(f"Created {len(chunks)} chunks")
        
        # Create vector store
        print("Creating vector store...")
        try:
            self.vectorstore = Chroma.from_documents(
                documents=chunks,
                embedding=self.embeddings,
                persist_directory=self.db_name
            )
            print("Vector store created successfully")
        except Exception as e:
            raise RuntimeError(f"Failed to create vector store: {e}")
        
        # Create retriever with MMR for better diversity
        self.retriever = self.vectorstore.as_retriever(
            search_type="mmr",  # Maximal Marginal Relevance
            search_kwargs={
                "k": self.k_docs,
                "fetch_k": self.k_docs * 2  # Fetch more, then rerank
            }
        )

        # result = self.vectorstore._collection.get(include=['embeddings', 'documents', 'metadatas'])
        # vectors = np.array(result['embeddings'])
        # documents = result['documents']
        # doc_types = [metadata['file_name'] for metadata in result['metadatas']]
        # colors = [['blue', 'green', 'red', 'orange'][['products', 'employees', 'contracts', 'company'].index(t)] for t in doc_types]
        # tsne = TSNE(n_components=2, random_state=42)
        # reduced_vectors = tsne.fit_transform(vectors)

        # # Create the 2D scatter plot
        # fig = go.Figure(data=[go.Scatter(
        #     x=reduced_vectors[:, 0],
        #     y=reduced_vectors[:, 1],
        #     mode='markers',
        #     marker=dict(size=5, color=colors, opacity=0.8),
        #     text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
        #     hoverinfo='text'
        # )])

        # fig.update_layout(
        #     title='2D Chroma Vector Store Visualization',
        #     scene=dict(xaxis_title='x',yaxis_title='y'),
        #     width=800,
        #     height=600,
        #     margin=dict(r=20, b=10, l=10, t=40)
        # )

        # fig.show()
    
    def answer_question(
        self,
        question: str,
        show_sources: bool = True,
        verbose: bool = False
    ) -> dict:
        """
        Answer a question based on the loaded PDF.
        
        Args:
            question: User's question
            show_sources: Whether to include source information
            verbose: Whether to print retrieved documents
            
        Returns:
            Dictionary with answer and optional metadata
        """
        if self.retriever is None:
            return {
                "answer": "Error: No document loaded. Please load a PDF first.",
                "sources": []
            }
        
        # Retrieve relevant documents
        try:
            docs = self.retriever.invoke(question)
        except Exception as e:
            return {
                "answer": f"Error retrieving documents: {e}",
                "sources": []
            }
        
        if verbose:
            print(f"\nRetrieved {len(docs)} documents:")
            for i, doc in enumerate(docs):
                print(f"\n--- Document {i+1} ---")
                print(doc.page_content[:200] + "...")
        
        if not docs:
            return {
                "answer": "I don't know. No relevant context found in the document.",
                "sources": []
            }
        
        # Prepare context
        context = "\n\n".join([
            f"[Excerpt {i+1}]:\n{doc.page_content}"
            for i, doc in enumerate(docs)
        ])
        
        # Create prompt
        prompt = f"""You are a helpful assistant that answers questions strictly based on the provided context from a book.
                CONTEXT:
                {context}

                QUESTION:
                {question}

                INSTRUCTIONS:
                - Answer the question using ONLY information from the context above
                - Be specific and cite relevant details from the context
                - If the answer cannot be found in the context, respond with: "I don't know - this information is not in the provided excerpts."
                - Do not make assumptions or add information not present in the context

                ANSWER:"""
        
        # Get LLM response
        try:
            response = self.llm.invoke([HumanMessage(content=prompt)])
            answer = response.content.strip()
        except Exception as e:
            return {
                "answer": f"Error generating answer: {e}",
                "sources": []
            }
        
        # Prepare result
        result = {"answer": answer}
        
        if show_sources:
            sources = []
            for i, doc in enumerate(docs):
                sources.append({
                    "chunk_id": doc.metadata.get("chunk_id", "unknown"),
                    "preview": doc.page_content[:150] + "...",
                    "filename": doc.metadata.get("filename", "unknown")
                })
            result["sources"] = sources
        
        return result


def main():
    """Example usage of the RAG system."""
    
    # Configuration
    subtle_art = "C:/Users/Paing/Downloads/fcuk.pdf"
    g_owell = "C:/Users/Paing/Downloads/1984.pdf"

    # Initialize RAG system
    rag_system = RAGBookQA(
        db_name="vector_db",
        chunk_size=1000,
        chunk_overlap=200,
        k_docs=4
    )
    
    # Load and process PDF (set force_reload=True to rebuild vector store)
    print("\n" + "="*50)
    print("Loading PDF and creating vector store...")
    print("="*50)
    
    try:
        rag_system.load_and_process_pdf(subtle_art, force_reload=False)
    except Exception as e:
        print(f"Error loading PDF: {e}")
        return
    
    # Ask questions
    questions = [
        "What is the first chapter of subtle art of not giving a f about?",
        "Who is the author of 1984?",
        "What is the name of the Second Lieutenant of Japanese Imperial Army in subtle art of not giving a f?"
    ]
    
    print("\n" + "="*50)
    print("Asking Questions")
    print("="*50)
    
    for question in questions:
        print(f"\n{'='*50}")
        print(f"Q: {question}")
        print(f"{'='*50}")
        
        result = rag_system.answer_question(
            question,
            show_sources=True,
            verbose=False
        )
        
        print(f"\nA: {result['answer']}")
        
        if result.get('sources'):
            print(f"\n📚 Sources used ({len(result['sources'])} excerpts):")
            for i, source in enumerate(result['sources']):
                print(f"\n  [{i+1}] Chunk {source['chunk_id']}: {source['preview']}")


if __name__ == "__main__":
    main()

c:\Users\paing\anaconda3\envs\llms\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading embedding model...
Initializing LLM...

Loading PDF and creating vector store...
Processing PDF: C:/Users/Paing/Downloads/fcuk.pdf
Splitting document into chunks...
Created 417 chunks
Creating vector store...
Vector store created successfully
Loading existing vector store from vector_db...

Asking Questions

Q: What is the first chapter of subtle art of not giving a f about?

A: I don't know - this information is not in the provided excerpts. The text does not explicitly mention a chapter title, and the excerpts appear to be an introduction to the concept of "not giving a f about" rather than a structured book with chapters.

📚 Sources used (4 excerpts):

  [1] Chunk 30: There’s a name for a person who finds no emotion or meaning in anything: a psychopath. Why you would want to emulate a psychopath, I have no fucking c...

  [2] Chunk 38: It then follows that finding something important and meaningful in your life is perhaps the most productive use of your time and energy. Beca